<a href="https://colab.research.google.com/github/anovaski/ToolsForStockData/blob/main/Stock_Correlations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/anovaski/mplchart.git

In [ ]:
!git clone https://github.com/anovaski/ToolsForStockData.git

Cloning into 'get_intraday_stock_data'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 32 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (32/32), 9.54 KiB | 1.91 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
import yfinance as yf
import pandas as pd
import statistics
import numpy as np

def read_tickers(textfile):
  textfile = open(textfile, 'r')
  textfile = textfile.read()
  textfile = textfile.split("\n")
  if textfile[-1] == "":
    textfile = textfile[0:len(textfile)-1]
  return textfile

spy = yf.Ticker("SPY")
spy_hist = spy.history(period="1y")
spy_dataframe = pd.DataFrame.from_dict(spy_hist)
spy_dataframe = spy_dataframe.pct_change()
spy_dataframe.drop(['Volume','Dividends','Stock Splits'], axis=1)

qqq = yf.Ticker("QQQ")
qqq_hist = qqq.history(period="1y")
qqq_dataframe = pd.DataFrame.from_dict(qqq_hist)
qqq_dataframe = qqq_dataframe.pct_change()
qqq_dataframe.drop(['Volume','Dividends','Stock Splits'], axis=1)

ibb = yf.Ticker("IBB")
ibb_hist = ibb.history(period="1y")
ibb_dataframe = pd.DataFrame.from_dict(ibb_hist)
ibb_dataframe = ibb_dataframe.pct_change()
ibb_dataframe.drop(['Volume','Dividends','Stock Splits'], axis=1)

#tickers = ['AAPL', 'MSFT', 'DGX']
tickers = read_tickers('/content/ToolsForStockData/ticker-lists/followed_tickers.txt')

ticker_list = []
beta_list = []
spy_list = []
qqq_list = []
ibb_list = []

for ticker in tickers:
  stock = yf.Ticker(ticker)
  try:
    beta = stock.info['beta']
  except:
    beta = 0
  stock_hist = stock.history(period="1y")
  stock_dataframe = pd.DataFrame.from_dict(stock_hist)
  stock_dataframe = stock_dataframe.pct_change()
  stock_dataframe.drop(['Volume','Dividends','Stock Splits'], axis=1)

  ticker_list.append(ticker)
  beta_list.append(beta)
  spy_list.append(spy_dataframe['Close'].corr(stock_dataframe['Close']))
  qqq_list.append(qqq_dataframe['Close'].corr(stock_dataframe['Close']))
  ibb_list.append(ibb_dataframe['Close'].corr(stock_dataframe['Close']))

data = {'ticker': ticker_list,
        'S&P500 Correlation': spy_list,
        'Nasdaq Correlation': qqq_list,
        'IBB Correlation': ibb_list,
        'Beta': beta_list,
        }
df = pd.DataFrame(data)
df.round(2)
print(df)
df.to_csv('followed_tickers_correlations.csv', index=False)

In [ ]:
sector_list = [
    'aesthetics',
    'biologics',
    'cardio',
    'cro',
    'diabetes',
    'diagnostic',
    'imaging',
    'labs',
    'lifesciequip',
    'lifescigenome',
    'meddevice',
    'neuro',
    'ocular',
    'oral',
    'pharma',
    'robotic',
    'spine',
    'variousdevice'
]

import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

for sector in sector_list:
  tickers = read_tickers('/content/ToolsForStockData/ticker-lists/get_intraday_stock_data/' + sector + '.txt')

  data = yf.download(tickers, period='20d', group_by='ticker')

  close_prices = pd.DataFrame()
  for ticker in tickers:
      close_prices[ticker] = data[ticker]['Close']

  # compute the correlation matrix
  corr_matrix = close_prices.corr()

  # create a heatmap of the correlation matrix using Seaborn
  sns.set(style='white')
  fig, ax = plt.subplots(figsize=(10, 10))
  sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', vmin=-1, vmax=1, ax=ax)

  # set the axis labels and title
  ax.set_xlabel('Stock Tickers')
  ax.set_ylabel('Stock Tickers')
  ax.set_title('10x10 Correlation Matrix of Stock Prices (20-day period)')

  # display the plot
  plt.savefig(sector + '.png', format='PNG')
  plt.show()